# 1. Dados do aluno

Jackes Tiago Ferreira da Fonseca - 190030291

# 2. Objetivo

Criar um modelo aplicando conceitos de NLP (Natural Language Processing) utilizando a biblioteca Transformers do HuggingFace, e participar de uma competição na plataforma Kaggle

# 3. Motivação

O uso de NLP tem crescido bastante bla bla, colocar refrerência da competição

# Modelagem

In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 7.8 MB/s eta 0:00:00


In [ ]:
#!pip uninstall transformers

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip uninstall sentencepiece

Found existing installation: sentencepiece 0.1.97
Uninstalling sentencepiece-0.1.97:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/sentencepiece-0.1.97.dist-info/*
    /usr/local/lib/python3.8/dist-packages/sentencepiece/*
Proceed (y/n)? 
Your response ('') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled sentencepiece-0.1.97


In [ ]:
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 326.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 310.5 MB/s eta 0:00:00


In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"jackesdafonseca","key":"abba8b28c5f2648d5d60f68049392137"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

Baixar os arquivos de dados da competição no Kaggle

In [ ]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 105MB/s]


In [ ]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from huggingface_hub import notebook_login
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TrainingArguments,Trainer
from datasets import Dataset,DatasetDict
import numpy as np 
import pandas as pd 
import kaggle, zipfile
import evaluate
import warnings

# remove avisos para o usuário
# warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
path = 'nlp-getting-started'

In [ ]:
kaggle.api.competition_download_cli(str(path))
zipfile.ZipFile(f'{path}.zip').extractall(path)

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls {path}

sample_submission.csv  test.csv  train.csv


Agora criamos um dataframe

In [ ]:
df = pd.read_csv(path + '/train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Para gerar uma descrição a fim de entender como cada coluna é usada podemos usar o método `describe()`

In [ ]:
df.describe(include='object')

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


In [ ]:
df.drop(columns = ['keyword', 'location'], inplace = True)
df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’

Geramos o dataset

In [ ]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'text', 'target'],
    num_rows: 7613
})

## Tokenização

Separação do texto em tokens

Para treinar o modelo utilizaremos 

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

`AutoTokenizer` irá criar um tokenizador apropriado para o modelo

In [ ]:
tokz = AutoTokenizer.from_pretrained(model_nm)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/a36c739020e01763fe789b4b85e2df55d6180012/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 0,
  "vocab_size": 128100
}



Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

loading file spm.model from cache at /root/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/a36c739020e01763fe789b4b85e2df55d6180012/spm.model
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/a36c739020e01763fe789b4b85e2df55d6180012/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--deberta-v3-small/snapshots/a36c739020e01763fe789b4b85e2df55d6180012/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  

Exemplo de frase separada em tokens

In [ ]:
tokz.tokenize("Damage to school bus on 80 in multi car crash #BREAKING")

O símbolo `_` serve para indicar o começo de uma nova palavra

A função a seguir tokeniza nossa entrada, e o map() executa a função em todas os itens do DataFrame.

In [ ]:
def tok_func(x): return tokz(x["text"])
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]


A seguir podemos ver um item do DataFrame, seu texto original e seus tokens gerados:

In [ ]:
row = tok_ds[0]
row['text'], row['input_ids']

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 [2,
  318,
  18055,
  50,
  14,
  1215,
  16,
  48,
  6926,
  12322,
  5418,
  1048,
  123,
  13,
  12209,
  8591,
  182,
  65,
  3])

In [ ]:
tokz.vocab['▁on']

27

A Transformers assume que nossos rótulos (valor que queremos prever) estão na coluna com nome labels, por isso precisamos renomear a nossa, que é a target.

In [ ]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

## Teste e validação

Para evitar overfitting, que é quando uma função fica ajustada demais ao conjunto de dados em que foi treinada e acaba desviando do formato real da função que estava sendo aproximada, é importante separar um conjunto de dados para não incluir no treinamento, pois não queremos que o modelo o veja.

Isso é chamado de conjunto de validação, e a biblioteca Transformers disponibiliza o train_test_split() para sua criação:

### Validação

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42) # 25% para validação e 75% para treino
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5709
    })
    test: Dataset({
        features: ['id', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1904
    })
})

### Teste



O conjunto de teste também é isento do treino, e só é usado após concluído todo o processo de treinamento.

A medida em que você tenta coisas diferentes para ver o impacto nas métricas do teste de validação, é possível que encontre algumas coisas que acidentalmente melhorem os resultados naquela situação, mas que no mundo real não são exatamente melhores. Por isso o conjunto de teste serve para mostrar se ocorreu overfitting com o conjunto de validação.


In [ ]:
eval_df = pd.read_csv(path + '/test.csv')
eval_df.drop(columns = ['keyword', 'location'], inplace = True)
eval_df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

## Treinando o modelo

Escolhemos um tamanho de lote (*batch size*) adequado à nossa GPU e um pequeno número de épocas para que possamos executar experimentos rapidamente. O hiperparâmetro mais importante é a taxa de aprendizado. fastai fornece um localizador de taxa de aprendizado para ajudá-lo a descobrir isso, mas Transformers não, então você só terá que usar tentativa e erro. A ideia é encontrar o maior valor possível, mas que não resulte em falha no treinamento.

In [ ]:
bs = 82
epochs = 4
lr = 8e-5

In [ ]:
args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine',
    evaluation_strategy="epoch", 
    per_device_train_batch_size=bs, 
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, 
    weight_decay=0.01, 
    report_to='none')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load('f1')
    logits, labels = eval_preds
    predictions = np.clip(logits, 0, 1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(
    model, 
    args, 
    train_dataset=dds['train'], 
    eval_dataset=dds['test'],
    tokenizer=tokz, 
    compute_metrics=compute_metrics
    )

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [ ]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5709
  Num Epochs = 4
  Instantaneous batch size per device = 82
  Total train batch size (w. parallel, distributed & accumulation) = 82
  Gradient Accumulation steps = 1
  Total optimization steps = 280
  Number of trainable parameters = 11684353
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than u

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

# Conclusão